<a href="https://colab.research.google.com/github/AsafShai/FantasyAverageRanking/blob/main/FantasyAverageRankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-colab-selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
import google_colab_selenium as gs

driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **GET STANDINGS FROM ESPN**

In [ ]:
import time

url = "https://fantasy.espn.com/basketball/league/standings?leagueId=1790719451"
driver.get(url)

# Wait for JavaScript to load content (adjust as needed)
time.sleep(5)

# Scroll to trigger lazy loading (if applicable)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Get the rendered HTML
html_content = driver.page_source
driver.quit()

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content, 'html.parser')
classes = soup.find_all(class_='ResponsiveTable')


teamNamesSpans = soup.find_all(class_='teamName')
teamNames = [element.text.strip() for element in teamNamesSpans[:10]]

In [ ]:
categoriesTH = soup.find_all(class_='Table__TH')
categories = [element.text for element in categoriesTH[-13:-1] if element.text != "" and element.text != "All"]

In [ ]:
tableTR = soup.find_all(class_='Table__TR')
rowsValues = []
for tableRow in tableTR[-22:-12]:
  tds = tableRow.findAll()
  rowsValues.append([element.text for element in tds[::2]])

gps = [tableRow.findAll()[0].text for tableRow in tableTR[-10:]]

In [ ]:
fullRowValues = []
for teamName, rowValues, gp in zip(teamNames, rowsValues, gps):
  fullRow = [teamName] + rowValues + [gp]
  fullRowValues.append({category: value for category, value in zip(categories, fullRow)})

# **CURRENT STANDINGS**



In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(fullRowValues)
df[df.columns[1:]] = df[df.columns[1:]].apply(pd.to_numeric, errors='coerce')
df

,Team,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,GP
0,Four Horseman,0.4964,0.8213,764,2890,2002,405,232,8229,452
1,Got Five Team,0.4913,0.8147,876,2409,1859,434,391,7892,458
2,Dor's Dangerous Team,0.4859,0.8095,768,2927,1834,401,408,8080,431
3,Roy's Rowdy Team,0.4777,0.8189,847,2698,1819,451,294,8086,430
4,Asaf's Awesome Team,0.4963,0.7936,763,2836,1878,410,379,7922,435
5,Way Too Fat,0.5059,0.8059,642,2720,1673,476,342,8212,434
6,Dbunk your junk,0.4867,0.7860,746,2757,2083,398,310,8992,473
7,Clay's Competitive Team,0.4670,0.7942,834,2508,1960,414,221,8613,399
8,TheBean02124,0.4722,0.8284,768,2067,1721,360,358,7393,444
9,Yonatan's Hearty Team,0.4772,0.7776,765,2601,1534,367,256,7764,418


# **PER GAME AVERAGES**

In [ ]:
for col in df.columns[3:-1]:
    df[col] /= df['GP']
df = df.round(4)
df

,Team,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,GP
0,Four Horseman,0.4964,0.8213,1.6903,6.3938,4.4292,0.8960,0.5133,18.2058,452
1,Got Five Team,0.4913,0.8147,1.9127,5.2598,4.0590,0.9476,0.8537,17.2314,458
2,Dor's Dangerous Team,0.4859,0.8095,1.7819,6.7912,4.2552,0.9304,0.9466,18.7471,431
3,Roy's Rowdy Team,0.4777,0.8189,1.9698,6.2744,4.2302,1.0488,0.6837,18.8047,430
4,Asaf's Awesome Team,0.4963,0.7936,1.7540,6.5195,4.3172,0.9425,0.8713,18.2115,435
5,Way Too Fat,0.5059,0.8059,1.4793,6.2673,3.8548,1.0968,0.7880,18.9217,434
6,Dbunk your junk,0.4867,0.7860,1.5772,5.8288,4.4038,0.8414,0.6554,19.0106,473
7,Clay's Competitive Team,0.4670,0.7942,2.0902,6.2857,4.9123,1.0376,0.5539,21.5865,399
8,TheBean02124,0.4722,0.8284,1.7297,4.6554,3.8761,0.8108,0.8063,16.6509,444
9,Yonatan's Hearty Team,0.4772,0.7776,1.8301,6.2225,3.6699,0.8780,0.6124,18.5742,418


# **RANKINGS (BASED ON AVERAGES)**


In [ ]:
# Columns to exclude from ranking
exclude_columns = ['Team', 'GP', 'MOVES']

# Selecting columns for ranking (excluding the ones in exclude_columns)
columns_for_ranking = [col for col in df.columns[1:] if col not in exclude_columns]


ranked = df.copy()
# Applying rank to selected columns
ranked[columns_for_ranking] = df[columns_for_ranking].rank()
ranked['Total_Points'] = ranked.iloc[:, 1:-1].sum(axis=1)

ranked

,Team,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,GP,Total_Points
0,Four Horseman,9.0,9.0,3.0,8.0,9.0,4.0,1.0,3.0,452,46.0
1,Got Five Team,7.0,7.0,8.0,2.0,4.0,7.0,8.0,2.0,458,45.0
2,Dor's Dangerous Team,5.0,6.0,6.0,10.0,6.0,5.0,10.0,6.0,431,54.0
3,Roy's Rowdy Team,4.0,8.0,9.0,6.0,5.0,9.0,5.0,7.0,430,53.0
4,Asaf's Awesome Team,8.0,3.0,5.0,9.0,7.0,6.0,9.0,4.0,435,51.0
5,Way Too Fat,10.0,5.0,1.0,5.0,2.0,10.0,6.0,8.0,434,47.0
6,Dbunk your junk,6.0,2.0,2.0,3.0,8.0,2.0,4.0,9.0,473,36.0
7,Clay's Competitive Team,1.0,4.0,10.0,7.0,10.0,8.0,2.0,10.0,399,52.0
8,TheBean02124,2.0,10.0,4.0,1.0,3.0,1.0,7.0,1.0,444,29.0
9,Yonatan's Hearty Team,3.0,1.0,7.0,4.0,1.0,3.0,3.0,5.0,418,27.0
